In [1]:
import pandas as pd
import numpy as np


#### Loading data

##### google drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/final_project_itc/data.npy'
data = np.load(path, allow_pickle = True)

In [5]:
df = pd.DataFrame(data, columns = ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'])


##### keep 3 features and split data

In [6]:
df = df.loc[:,['ProductId', 'Text', 'Score']]

In [7]:
from sklearn.model_selection import GroupShuffleSplit
splitter_temp = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 42)
split_temp = splitter_temp.split(df[:100000], groups=df[:100000]['ProductId'])
train_inds, temp_inds = next(split_temp)

train = df.iloc[train_inds]
temp = df.iloc[temp_inds]


In [8]:
splitter_val = GroupShuffleSplit(test_size=.50, n_splits=1, random_state = 42)
split_val = splitter_val.split(temp, groups=temp['ProductId'])
val_inds, test_inds = next(split_val)

val = temp.iloc[val_inds]
test = temp.iloc[test_inds]

In [9]:
X_train = train.drop(columns = 'Score')
y_train = train.Score

X_val = val.drop(columns = 'Score')
y_val = val.Score

X_test = test.drop(columns = 'Score')
y_test = test.Score

#### Bert

##### install packages

In [10]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 784.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [11]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


##### Bert regression

In [12]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
base_model = 'bert-base-cased'
learning_rate = 2e-5
max_length = 64
batch_size = 16
epochs = 5
nbr_samples = 10000
tokenizer_regr = AutoTokenizer.from_pretrained(base_model)
model_regr = AutoModelForSequenceClassification.from_pretrained(base_model,num_labels = 1)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
X_train_bert = X_train.iloc[:nbr_samples]
del X_train_bert['ProductId']
X_train_bert['label'] = y_train.iloc[:nbr_samples].astype(float)

<ipython-input-13-c84a96c26338>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_bert['label'] = y_train.iloc[:nbr_samples].astype(float)


In [14]:
X_val_bert = X_val.iloc[:nbr_samples]
del X_val_bert['ProductId']
X_val_bert['label'] = y_val.iloc[:nbr_samples].astype(float)

X_test_bert = X_test.iloc[:nbr_samples]
del X_test_bert['ProductId']
X_test_bert['label'] = y_test.iloc[:nbr_samples].astype(float)

<ipython-input-14-f798007a74ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_bert['label'] = y_val.iloc[:nbr_samples].astype(float)
<ipython-input-14-f798007a74ee>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_bert['label'] = y_test.iloc[:nbr_samples].astype(float)


In [15]:
from datasets import Dataset
ds_train_regr = Dataset.from_pandas(X_train_bert)
ds_val_regr = Dataset.from_pandas(X_val_bert)
ds_test_regr = Dataset.from_pandas(X_test_bert)

In [16]:
def preprocess_function_regr(examples):
    return tokenizer_regr(examples["Text"], truncation=True, max_length=64, padding = 'max_length')

In [17]:
ds_train_regr_tok = ds_train_regr.map(preprocess_function_regr, remove_columns = ['Text'])
ds_val_regr_tok = ds_val_regr.map(preprocess_function_regr, remove_columns = ['Text'])
ds_test_regr_tok = ds_test_regr.map(preprocess_function_regr, remove_columns = ['Text'])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    # Compute accuracy
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)

    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [19]:
from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/"

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
) # Same as the classification

In [20]:
from transformers import Trainer
import torch
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [21]:
trainer = Trainer(
    model=model_regr,
    args=training_args,
    train_dataset=ds_train_regr_tok,
    eval_dataset=ds_val_regr_tok,
    compute_metrics=compute_metrics_for_regression
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,1.596200,0.853404,0.853404,0.648202,0.528595,0.538400
2,0.651700,0.847520,0.847520,0.547483,0.531845,0.647200
3,0.448900,0.842752,0.842752,0.542698,0.534479,0.653100
4,0.223800,0.846023,0.846023,0.530756,0.532672,0.658300
5,0.164300,0.844006,0.844006,0.527611,0.533786,0.663300


TrainOutput(global_step=3125, training_loss=0.5489909454345703, metrics={'train_runtime': 461.0626, 'train_samples_per_second': 108.445, 'train_steps_per_second': 6.778, 'total_flos': 1644429331200000.0, 'train_loss': 0.5489909454345703, 'epoch': 5.0})

In [22]:
trainer.eval_dataset = ds_test_regr_tok
eval_results = trainer.evaluate()

In [23]:
eval_results

{'eval_loss': 0.8059501051902771,
 'eval_mse': 0.8059501051902771,
 'eval_mae': 0.5098522305488586,
 'eval_r2': 0.5516140060425027,
 'eval_accuracy': 0.6767,
 'eval_runtime': 15.7264,
 'eval_samples_per_second': 635.874,
 'eval_steps_per_second': 39.742,
 'epoch': 5.0}

In [24]:
predictions = trainer.predict(ds_test_regr_tok)

In [25]:
predicted_labels = predictions.predictions

##### Checking high errors on labelling

In [27]:
df_with_labels = pd.DataFrame({'test_label':y_test.iloc[:nbr_samples],'predicted_labels':pd.Series(predicted_labels.flatten(), index = y_test.iloc[:nbr_samples].index)}, index = y_test.iloc[:nbr_samples].index)

In [28]:
df_with_labels[(df_with_labels.test_label > 4) & (df_with_labels.predicted_labels < 2)]

,test_label,predicted_labels
1300,5,1.571106
1460,5,1.938118
3738,5,1.871366
3968,5,1.289142
3989,5,1.616189
...,...,...
52514,5,1.425838
52527,5,1.064740
52533,5,1.425838
52537,5,1.060332


In [29]:
X_test.loc[1300].Text

'this is the second time ordering beer nuts....i can not find them anywhere in local stores.  the shipment came (6 cans per order) and 2 cans were damaged.  called amazon customer service and the next day another 6 cans appeared at my door.<br /><br />the nuts are always great (unless can is damaged and nuts are stale) but the customer service was the shining star.  thank you !!  will order again'

In [30]:
X_test.loc[52537].Text

'I have two litter mates.  One can eat anything with no problem.  The other always vomits ten minutes after eating, no matter what she eats: canned or dried makes no difference and I\'ve tried every store brand there is.  Hairball treatments had no effect.  I figured "cats throw up" and didn\'t worry about it.  When I moved and switched vets the new vet asked if the cats ever threw up.  When she heard about the one cat she said cats do NOT just throw up and told me to get the Science Diet sensitive stomach food.  Yes, it\'s 4x the price of what I\'d been buying, but I noticed a difference with the first bowl.  Instead of throwing up ten minutes after literally every meal, now she throws up maybe once a month.  If only I\'d switched to this brand years ago we both would have been saved a lot of misery.  Even tho it costs more I\'ve noticed that the bag seems to last longer than the same amount of cat chow did.'

In [31]:
X_test.loc[52558].Text

"The story goes that I adopted a cat from an agency last summer. Having always grown up with dogs and not cats, I went ahead and wandered down to my Target to get a bag of Friskies. About a week later, my kitty (named Sugar) starting vomiting. Confused, I assumed it was the wet food we were feeding her in the morning. So we changed it. Then we changed it again. No results. I took her to the vet and asked. He checked her out and said likely the problem was having a sensitive stomach and he recommended Science Diet. Bam. My cat does great now. She's lost a little bit of weight so she's the right size now. She doesn't vomit anymore. This food works and it works well. I read the negative review here and I would say the problem was unique to the cat. My cat loves it and is doing brilliantly now."